In [1]:
import pandas as pd
from pathlib import Path
import pylatex as pl
import pymc as pm
import arviz as az
import plotly.express as px
import numpy as np
import pandas as pd

from estival.model import BayesianCompartmentalModel
import estival.priors as esp
import estival.targets as est
from estival.wrappers import pymc as epm
from tbdynamics import model
from tbdynamics.inputs import fixed_parameters
from tbdynamics.utils import round_sigfig
import plotly.graph_objects as go

WARNING (pytensor.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (pytensor.configdefaults): g++ not detected!  PyTensor will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set PyTensor flags cxx to an empty string.
WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
PROJECT_PATH = Path().resolve()
DATA_PATH = PROJECT_PATH / 'data'

In [3]:
pd.options.plotting.backend = "plotly"
time_start = 1800
time_end = 2020
time_step = 0.1

doc_sections = {}
compartments = [
    "susceptible",
    "early_latent",
    "late_latent",
    "infectious",
    "on_treatment",
    "recovered",
]
infectious_compartments = [
    "infectious",
    "on_treatment",
]

latent_compartments = [
    "early_latent",
    "late_latent",
]
age_strata = [0,5,15,35,50,70] 

In [4]:
matrix = np.ones((6, 6))

In [5]:
tb_model = model.build_model(
    compartments,
    infectious_compartments,
    latent_compartments,
    age_strata,
    time_start,
    time_end,
    time_step,
    matrix, # change the matrix
    fixed_parameters,
    add_triangular= False # add boolean switch to infectious seed
)

In [6]:
params = {
    "contact_rate": 0.009414102898074345,
    "start_population_size": 23000000,
    "progression_multiplier": 1.1,
    "rr_infection_latent": 0.2,
    "rr_infection_recovered": 0.2,
    "seed_time": 1960,
    "seed_duration": 1,
}
priors = [
    esp.UniformPrior("start_population_size", (1000000, 30000000)),
    esp.UniformPrior("contact_rate", (0.1, 130)),  # multiplied with eigenvalue
    esp.UniformPrior("rr_infection_latent", (0.2, 0.5)),
    esp.UniformPrior("rr_infection_recovered", (0.1, 0.5)),
    esp.UniformPrior("progression_multiplier", (0.1, 10.0)),
    esp.UniformPrior("seed_time", [1800.0, 1920.0]),
    esp.UniformPrior("seed_duration", [1.0, 2.0]),
]


pop = pd.Series(
    {
        2000: 77630900,
        2001: 78620500,
        2002: 79537700,
        2003: 80467400,
        2004: 81436400,
        2005: 82392100,
        2006: 83311200,
        2007: 84218500,
        2008: 85118700,
        2009: 86025000,
        2010: 87067300,
        2011: 88145800,
        2012: 89202900,
        2013: 90191400,
        2014: 91203800,
        2015: 92228600,
        2016: 93250700,
        2017: 94286000,
        2018: 95385200,
        2019: 96484000,
        2020: 97582690,
        2021: 98504400,
    }
)
targets = [
    est.NormalTarget("total_population", pop, stdev=10000),
]
calibration_model = BayesianCompartmentalModel(tb_model, params, priors, targets)

In [7]:
with pm.Model() as pmc_model:
    start_params = {k: np.clip(v, *calibration_model.priors[k].bounds(0.99)) for k, v in params.items() if k in calibration_model.priors}
    variables = epm.use_model(calibration_model)
    map_params = pm.find_MAP(start=start_params, vars=variables, maxeval= 5000,include_transformed=False)
    map_params = {k: float(v) for k, v in map_params.items()}
    print('Best calibration parameters found:')
for i_param, param in enumerate(map_params):
    print(f'   {param}: {round_sigfig(map_params[param], 4)} (within bound {priors[i_param].bounds()}')


Best calibration parameters found:
   start_population_size: 29320000.0 (within bound (1000000.0, 30000000.0)
   contact_rate: 90.83 (within bound (10.0, 1300.0)
   rr_infection_latent: 0.2188 (within bound (0.2, 0.5)
   rr_infection_recovered: 0.4264 (within bound (0.1, 0.5)
   progression_multiplier: 6.031 (within bound (0.1, 10.0)
   seed_time: 1920.0 (within bound (1800.0, 1920.0)
   seed_duration: 1.063 (within bound (1.0, 2.0)


In [8]:
params.update(map_params)
tb_model.run(params)
derived_df_0 = tb_model.get_derived_outputs_df()

In [9]:
fig2_1 = px.line(
    derived_df_0,
    x=derived_df_0.index,
    y="total_population",
)
plots = {"total_population": {
      "title": "Population size",
      "output_key": "total_population",
      "times": [2009.0, 2019.0],
      "values": [86025000, 96484000],
      "quantiles": [0.025, 0.25, 0.5, 0.75, 0.975]
    },
     "notifications": {
      "title": "Notifications",
      "output_key": "notifications",
      "times": [2013.0, 2014.0, 2015.0, 2016.0, 2017.0, 2018.0, 2019.0, 2020.0, 2021.0, 2022.0],
      "values": [102196, 102087, 102676, 102527, 105733, 102171, 104505, 101795, 78935, 103804],
      "quantiles": [0.025, 0.25, 0.5, 0.75, 0.975]
    },
    "percentage_latent": {
      "title": "Percentage Latent",
      "output_key": "percentage_latent",
      "times": [2016.0],
      "values": [30.8],
      "quantiles": [0.025, 0.25, 0.5, 0.75, 0.975]
    },
    }
fig2_2 = px.scatter(x= plots['total_population']['times'], y = plots['total_population']['values'])
fig2_2.update_traces(marker=dict(color="red"))
fig2_3 = go.Figure(
    data=fig2_1.data + fig2_2.data,
)
fig2_3.update_layout(
    title="Modelled vs Data", title_x=0.5, xaxis_title="Year", yaxis_title="Population"
)
fig2_3.show()
